## Сравнение коридора цен ЗУ на торгах и предложений на авито

In [404]:
import pickle
import pandas as pd
import numpy as np
from statistics import mean

### 1. Загрузим, подготовим и сгруппируем данные по торгам

In [3]:
'Десериализуем последнюю версию датафрейма с лотами из pickle файла'

pkl_data_path = '/Users/dmitrijdolgopolov/Documents/1_study_python/projects/' +\
                '9_torgi.gov_api/landplots_lots_stats/1_parser/lots.pkl'

with open(pkl_data_path, 'rb') as file:
    lots_df = pickle.load(file)

**1.1. Исключим лоты с отсутствующей ключевой информацией (цена, площадь)**

In [41]:
lots_df = lots_df.query('end_price != "Неизвестно" & kad_cost != "Неизвестно" & start_price < end_price \
                        & kad_cost != 0')

In [103]:
lots_df.shape

(27498, 14)

In [53]:
lots_df.head(2)

,link,kad_num,region,district,category,vri,contract,square,start_price,end_price,date,kad_cost,price_increase,end_price_sotka
0,https://torgi.gov.ru/new/public/lots/lot/21000...,90:07:200301:1533,Крым,Ленинский муниципальный район,Земли населенных пунктов,Для индивидуального жилищного строительства,Аренда,1000.0,73746.0,109144.08,2022-12-05,725650.0,48.0,10914.0
1,https://torgi.gov.ru/new/public/lots/lot/21000...,90:07:200101:3091,Крым,Ленинский муниципальный район,Земли населенных пунктов,Для индивидуального жилищного строительства,Аренда,1178.0,90541.0,679962.91,2022-12-05,2359705.6,651.0,57722.0


In [171]:
# lots_df.category.value_counts()

**1.2. Проверим типы данных в колонках и приведем к нужным**

In [99]:
# lots_df.dtypes

In [15]:
lots_df.end_price = lots_df.end_price.astype('float')

In [20]:
lots_df.kad_cost = lots_df.kad_cost.astype('float')

In [25]:
lots_df.date = pd.to_datetime(lots_df.date)

**1.3. Группировка и получение итоговых таблиц по лотам**

In [48]:
'Добавим колонку с процентом увеличения конечной цены относительно стартовой'

lots_df['price_increase'] = np.round(100 * (lots_df.end_price - lots_df.start_price) / lots_df.start_price, 2)

In [51]:
'Добавим колонку с конечной ценой за сотку'

lots_df['end_price_sotka'] = np.round(lots_df.end_price / (lots_df.square / 100), 0)

*1.3.1. Соберем сводные таблицы: Участки под жилье (отдельно в аренду, отдельно - покупка)*

In [54]:
'Список интересующих ВРИ'

house_building_vri_list = ['Для индивидуального жилищного строительства',
                            'Для ведения личного подсобного хозяйства (приусадебный земельный участок)',
                           'Жилая застройка',
                           'Блокированная жилая застройка',
                           'Малоэтажная многоквартирная жилая застройка',
                           'Ведение садоводства',
                           'Среднеэтажная жилая застройка']

In [79]:
'Только участки под жилье, тип договора - АРЕНДА'

land_for_living_lots_rent_df = lots_df.query('vri in @house_building_vri_list & contract == "Аренда"')

In [80]:
'Только участки под жилье, тип договора - ПРОДАЖА'

land_for_living_lots_buy_df = lots_df.query('vri in @house_building_vri_list & contract == "Продажа"')

In [81]:
land_for_living_lots_rent_df.shape

(10037, 14)

In [82]:
land_for_living_lots_buy_df.shape

(3873, 14)

In [86]:
print(f"Доля участков для жилья от общего количества лотов: ", 
      f"""{np.round(100 * (land_for_living_lots_buy_df.shape[0] + 
        land_for_living_lots_rent_df.shape[0])/lots_df.shape[0], 0)}%""")

Доля участков для жилья от общего количества лотов:  51.0%


In [343]:
'Сгруппируем ЗУ по региону, категории и типу торгов. Тип торгов: АРЕНДА. Оставим регионы с более 19 лотами'

pivot_living_rent = land_for_living_lots_rent_df.groupby(['region', 'category', 'contract'], as_index=False) \
            .agg(lot_count=("link", "count"),
                q25_lot_pr_incr=("price_increase", lambda x: np.quantile(x, 0.25).round(0)),
                median_lot_pr_incr=("price_increase", lambda x: np.median(x).round(0)),
                q75_lot_pr_incr=("price_increase", lambda x: np.quantile(x, 0.75).round(0)),
                q25_lot_end_pr_sot=("end_price_sotka", lambda x: np.quantile(x, 0.25).round(0)),
                median_lot_end_pr_sot=("end_price_sotka", lambda x: np.median(x).round(0)),
                q75_lot_end_pr_sot=("end_price_sotka", lambda x: np.quantile(x, 0.75).round(0))) \
            .sort_values('median_lot_end_pr_sot', ascending=False) \
            .query('lot_count >= 20')

In [375]:
'Сгруппируем ЗУ по региону, категории и типу торгов. Тип торгов: ПРОДАЖА. Оставим регионы с более 9 лотами'

pivot_living_sale = land_for_living_lots_buy_df.groupby(['region', 'category', 'contract'], as_index=False) \
            .agg(lot_count=("link", "count"),
                q25_lot_pr_incr=("price_increase", lambda x: np.quantile(x, 0.25).round(0)),
                median_lot_pr_incr=("price_increase", lambda x: np.median(x).round(0)),
                q75_lot_pr_incr=("price_increase", lambda x: np.quantile(x, 0.75).round(0)),
                q25_lot_end_pr_sot=("end_price_sotka", lambda x: np.quantile(x, 0.25).round(0)),
                median_lot_end_pr_sot=("end_price_sotka", lambda x: np.median(x).round(0)),
                q75_lot_end_pr_sot=("end_price_sotka", lambda x: np.quantile(x, 0.75).round(0))) \
            .sort_values('median_lot_end_pr_sot', ascending=False) \
            .query('lot_count >= 10')

In [345]:
# pivot_living_rent.head(5)

In [376]:
pivot_living_sale.shape

(67, 10)

*1.3.2. Соберем сводные таблицы: Сельхозка (отдельно в аренду, отдельно - покупка). Из сельхоз земель исключим ВРИ "Ведение садоводства", так как он уже отнесен к землям для жилья*

In [110]:
'Список исключаемых ВРИ'

exclude_vri_list = ['Ведение садоводства']

In [112]:
'Только сельхозка, тип договора - АРЕНДА'

selhozka_lots_rent_df = lots_df.query('category == "Земли сельскохозяйственного назначения" \
                                        & vri not in @house_building_vri_list & contract == "Аренда"')

In [114]:
'Только сельхозка, тип договора - ПРОДАЖА'

selhozka_lots_sale_df = lots_df.query('category == "Земли сельскохозяйственного назначения" \
                                        & vri not in @house_building_vri_list & contract == "Продажа"')

In [116]:
selhozka_lots_rent_df.shape

(3785, 14)

In [117]:
selhozka_lots_sale_df.shape

(627, 14)

In [118]:
print(f"Доля участков для жилья от общего количества лотов: ", 
      f"""{np.round(100 * (selhozka_lots_rent_df.shape[0] + 
        selhozka_lots_sale_df.shape[0])/lots_df.shape[0], 0)}%""")

Доля участков для жилья от общего количества лотов:  16.0%


In [341]:
'Сгруппируем ЗУ по региону, категории и типу торгов. Тип торгов: АРЕНДА. Оставим регионы с более 4 лотами'

pivot_selhozka_rent = selhozka_lots_rent_df.groupby(['region', 'category', 'contract'], as_index=False) \
            .agg(lot_count=("link", "count"),
                q25_lot_pr_incr=("price_increase", lambda x: np.quantile(x, 0.25).round(0)),
                median_lot_pr_incr=("price_increase", lambda x: np.median(x).round(0)),
                q75_lot_pr_incr=("price_increase", lambda x: np.quantile(x, 0.75).round(0)),
                q25_lot_end_pr_sot=("end_price_sotka", lambda x: np.quantile(x, 0.25).round(0)),
                median_lot_end_pr_sot=("end_price_sotka", lambda x: np.median(x).round(0)),
                q75_lot_end_pr_sot=("end_price_sotka", lambda x: np.quantile(x, 0.75).round(0))) \
            .sort_values('median_lot_end_pr_sot', ascending=False) \
            .query('lot_count >= 5')

In [340]:
'Сгруппируем ЗУ по региону, категории и типу торгов. Тип торгов: ПРОДАЖА. Оставим регионы с более 4 лотами'

pivot_selhozka_sale = selhozka_lots_sale_df.groupby(['region', 'category', 'contract'], as_index=False) \
            .agg(lot_count=("link", "count"),
                q25_lot_pr_incr=("price_increase", lambda x: np.quantile(x, 0.25).round(0)),
                median_lot_pr_incr=("price_increase", lambda x: np.median(x).round(0)),
                q75_lot_pr_incr=("price_increase", lambda x: np.quantile(x, 0.75).round(0)),
                q25_lot_end_pr_sot=("end_price_sotka", lambda x: np.quantile(x, 0.25).round(0)),
                median_lot_end_pr_sot=("end_price_sotka", lambda x: np.median(x).round(0)),
                q75_lot_end_pr_sot=("end_price_sotka", lambda x: np.quantile(x, 0.75).round(0))) \
            .sort_values('median_lot_end_pr_sot', ascending=False) \
            .query('lot_count >= 5')

In [140]:
# pivot_selhozka_rent.head(10)

In [141]:
# pivot_selhozka_sale.head(10)

### 2. Загрузим, подготовим и сгруппируем данные по Авито

In [271]:
'Десериализуем последнюю версию датафрейма с объявлениями Авито из pickle файла'

pkl_avito_data_path = '/Users/dmitrijdolgopolov/Documents/1_study_python/projects/' +\
                      '9_torgi.gov_api/new_parser_avito/1_parser/ads_landplots.pkl'

with open(pkl_avito_data_path, 'rb') as file:
    avito_df = pickle.load(file)

**2.1. Исключим объявления с отсутствующей ключевой информацией (цена, площадь)**

In [272]:
'Добавим колонку с ценой за сотку'

avito_df['price_sotka'] = np.round(avito_df.price / avito_df.square, 0)

In [273]:
avito_df = avito_df[avito_df.price != 0]

In [274]:
avito_df = avito_df[avito_df.land_cat != '-']

In [277]:
np.quantile(avito_df.price_sotka, 0.009)

78.0

In [278]:
'Исключим объявления с экстремально низкой или высокой ценой сотки участка'

price_sotka_q0_009 = np.quantile(avito_df.price_sotka, 0.009)
avito_df = avito_df.query('price_sotka < 50000000').query('price_sotka > @price_sotka_q0_03')

In [327]:
# print(avito_df.loc[666, 'link'])

**2.2. Проверим типы данных в колонках и приведем к нужным**

In [239]:
# avito_df.dtypes

**2.3. Группировка и получение итоговых таблиц по Авито**

*2.3.1. Переименуем наименование категорий в соответствии с таблицей лотов*

In [279]:
'Создадим словарь соответствий названий категорий в авито и лотах'

land_cat_dict = {'Поселений (ИЖС)': 'Земли населенных пунктов',
                'Сельхозназначения (СНТ, ДНП)': 'Земли сельскохозяйственного назначения'}

In [280]:
'Переименуем категории в соответствии со словарем land_cat_dict'

avito_df.land_cat = avito_df.land_cat \
                            .apply(lambda x: land_cat_dict[x.strip()] if x.strip() in land_cat_dict else x)

In [193]:
# 'Заполним отсутствующие значения land_cat'

# for i in avito_df[avito_df.land_cat == '-']:
#     if 'личное подсобное хозяйство' in avito_df.loc[i, 'title'] and avito_df.loc[i, 'square'] < 200:
#         avito_df.loc[i, 'title']

In [243]:
# avito_df.land_cat.value_counts()

In [179]:
# lots_df.category.value_counts()

*2.3.2. Соберем сводные таблицы: Участки под **ЖИЛЬЕ** (имеем только участки для продажи)*

In [283]:
# avito_df.shape

In [287]:
'1. Сельхоз участки под ведение садоводства и дачное строительство (примем, что это участки площадью < 200 сот)'

avito_add_living_p1 = avito_df.query("land_cat == 'Земли сельскохозяйственного назначения' & square < 200")

In [306]:
'2. Участки в населенном пункте'

avito_add_living_p2 = avito_df.query("land_cat == 'Земли населенных пунктов'")

In [309]:
'Объединим обе части в общую таблицу (1+2)'

avito_add_living = pd.concat([avito_add_living_p1, avito_add_living_p2])

In [312]:
'Проверка правильности конкатенации'

# avito_add_living_p1.shape[0] + avito_add_living_p2.shape[0] == avito_add_living.shape[0]

True

In [314]:
print(f"Доля участков для жилья от общего количества объявлений: ", 
      f"""{np.round(100 * (avito_add_living.shape[0] / avito_df.shape[0]), 0)}%""")

Доля участков для жилья от общего количества объявлений:  93.0%


In [317]:
avito_add_living.head(1)

,add_id,link,title,price,land_cat,square,ads_time,person_type,region,city,address,description,nedvigimost_type,coords,location,time_source_created,time_source_updated,is_actual,district,price_sotka
1,623819209,https://www.avito.ru/uvarovka/zemelnye_uchastk...,"Участок 12 сот. (СНТ, ДНП)",199999,Земли сельскохозяйственного назначения,12.0,2024-02-29 17:45:26,Частное лицо (фильтр),Московская область,Уваровка,"Московская обл., Можайский г.о., коттеджный по...",В будние дни звонить ПОСЛЕ 18:00. До этого вре...,Продам,"55.66884943217913, 35.57753456303382",-,2023-11-24 14:51:40,2024-02-22 14:58:27,11,Можайский Городской Округ,16667.0


In [338]:
'Сгруппируем ЗУ по региону, категории и типу торгов. Оставим регионы с более 100 лотами'

pivot_avito_living =  avito_add_living.groupby(['region', 'land_cat'], as_index=False) \
            .agg(avito_add_count=("link", "count"),
                q25_avito_pr_sot=("price_sotka", lambda x: np.quantile(x, 0.25).round(0)),
                median_avito_pr_sot=("price_sotka", lambda x: np.median(x).round(0)),
                q75_avito_pr_sot=("price_sotka", lambda x: np.quantile(x, 0.75).round(0))) \
            .sort_values('median_avito_pr_sot', ascending=False) \
            .query('avito_add_count >= 100')

*2.3.2. Соберем сводные таблицы: Участки под **СЕЛЬХОЗКУ** (имеем только участки для продажи)*

In [329]:
avito_add_selhozka = avito_df.query("land_cat == 'Земли сельскохозяйственного назначения' & square > 200")

In [330]:
print(f"Доля участков для сельхозки от общего количества объявлений: ", 
      f"""{np.round(100 * (avito_add_selhozka.shape[0] / avito_df.shape[0]), 0)}%""")

Доля участков для сельхозки от общего количества объявлений:  4.0%


In [337]:
'Сгруппируем ЗУ по региону, категории и типу торгов. Оставим регионы с более 100 лотами'

pivot_avito_selhozka = avito_add_selhozka.groupby(['region', 'land_cat'], as_index=False) \
            .agg(avito_add_count=("link", "count"),
                q25_avito_pr_sot=("price_sotka", lambda x: np.quantile(x, 0.25).round(0)),
                median_avito_pr_sot=("price_sotka", lambda x: np.median(x).round(0)),
                q75_avito_pr_sot=("price_sotka", lambda x: np.quantile(x, 0.75).round(0))) \
            .sort_values('median_avito_pr_sot', ascending=False) \
            .query('avito_add_count >= 20')

### 3. Совместим данные по торгам и Авито (привязка: регион и категория земли)

**3.1. ЖИЛЬЕ**

*3.1.1. Соединение таблиц*

In [353]:
# pivot_living_sale.head(1)

In [377]:
'Таблица лотов (только с нужными клонками)'

pivot_living_lots = pivot_living_sale[pivot_living_sale\
                                      .columns.drop(['contract', 'q25_lot_pr_incr', 'q75_lot_pr_incr'])]

In [359]:
'Таблица Авито мерджится в исходном виде, но с переименованием колонки land_cat в category'

pivot_avito_living = pivot_avito_living.rename(columns={'land_cat': 'category'})

In [365]:
pivot_living_lots.shape

(47, 7)

In [366]:
pivot_avito_living.shape

(150, 6)

In [388]:
# pivot_living_lots[['region', 'category', 'lot_count']]

In [392]:
'Соединяем две таблицы. ИТОГОВАЯ ТАБЛИЦА СРАВНЕНИЯ ЛОТОВ И АВИТО'

merged_pivot_living = pivot_living_lots.merge(pivot_avito_living, on=['region', 'category'], how='inner')

*3.1.2. Анализ, добавление агрегированных колонок и сегментация регионов*

In [408]:
'Добавим новую колонку для среднего значения превышения цен авито над итоговыми ценами лотов'

merged_pivot_living['avg_price_diff'] = 0.0

In [423]:
'Посчитаем среднее значение превышения цен авито над итоговыми ценами лотов (среднее между q25, median, q75)'

for i in merged_pivot_living.index:
    merged_pivot_living.loc[i, 'avg_price_diff'] = 100 * np.mean(\
     [merged_pivot_living.loc[i, 'q25_avito_pr_sot'] / merged_pivot_living.loc[i, 'q25_lot_end_pr_sot']-1,
     merged_pivot_living.loc[i, 'median_avito_pr_sot'] / merged_pivot_living.loc[i, 'median_lot_end_pr_sot']-1,
     merged_pivot_living.loc[i, 'q75_avito_pr_sot'] / merged_pivot_living.loc[i, 'q75_lot_end_pr_sot']-1]
                                                                ).round(2)


In [438]:
'Посчитаем значения квантилей (0.25, 0.5, 0.75) для основных сравниваемых колонок'

lc_quantile = list(merged_pivot_living.lot_count.quantile([0.25, 0.5, 0.75]))
mlpi_quantile = list(merged_pivot_living.median_lot_pr_incr.quantile([0.25, 0.5, 0.75]))
aac_quantile = list(merged_pivot_living.avito_add_count.quantile([0.25, 0.5, 0.75]))
apd_quantile = list(merged_pivot_living.avg_price_diff.quantile([0.25, 0.5, 0.75]))

In [439]:
'Для присвоения баллов по каждому показателю сделаем функции'

def lc_group(x, lc_quantile):
    if x < lc_quantile[0]:
        return int(1)
    elif x < lc_quantile[1]:
        return int(2)
    elif x < lc_quantile[2]:
        return int(3)
    elif x >= lc_quantile[2]:
        return int(4)
    
    
def mlpi_group(x, mlpi_quantile):
    if x < mlpi_quantile[0]:
        return int(4)
    elif x < mlpi_quantile[1]:
        return int(3)
    elif x < mlpi_quantile[2]:
        return int(2)
    elif x >= mlpi_quantile[2]:
        return int(1)
    
    
def aac_group(x, aac_quantile):
    if x < aac_quantile[0]:
        return int(1)
    elif x < aac_quantile[1]:
        return int(2)
    elif x < aac_quantile[2]:
        return int(3)
    elif x >= aac_quantile[2]:
        return int(4)
    
    
def apd_group(x, apd_quantile):
    if x < apd_quantile[0]:
        return int(1)
    elif x < apd_quantile[1]:
        return int(2)
    elif x < apd_quantile[2]:
        return int(3)
    elif x >= apd_quantile[2]:
        return int(4)

In [440]:
'Для наглядности проставим баллы по основным сравниваемым колонкам по аналогии с RFM-сегментацией'

'Сегм. по avg_price_diff: 1-4 - от меньш. к больш. разнице цен авито к ценам лотов'
merged_pivot_living['APD'] = merged_pivot_living.avg_price_diff.apply(lambda x: apd_group(x, apd_quantile))

'Сегм. по median_lot_pr_incr: 1-4 - от больш. к меньш. знач. медианы увелич. цены'
merged_pivot_living['MLPI'] = merged_pivot_living.median_lot_pr_incr.apply(lambda x: mlpi_group(x, mlpi_quantile))

'Сегм. по lot_count: 1-4 - от меньшего к большему количеству лотов'
merged_pivot_living['LC'] = merged_pivot_living.lot_count.apply(lambda x: lc_group(x, lc_quantile)) 

'Сегм. по avito_add_count: 1-4 - от меньш. к больш. количеству объявл. на авито'
merged_pivot_living['AAC'] = merged_pivot_living.avito_add_count.apply(lambda x: aac_group(x, aac_quantile))  

In [443]:
'Сортируем итоговую таблицу по баллам'

merged_pivot_living = merged_pivot_living.sort_values(['APD', 'MLPI', 'LC', 'AAC'], 
                                                      ascending=[False for _ in range(4)])

In [444]:
'Сохранение итоговой таблицы в csv'

merged_pivot_living.to_csv('lots_and_avito_stats_by_region.csv', index=False)

### 4. Сводные данные по ценам участков под жилье на Авито с группировкой по регионам и районам

In [447]:
avito_add_living.head(1)

,add_id,link,title,price,land_cat,square,ads_time,person_type,region,city,address,description,nedvigimost_type,coords,location,time_source_created,time_source_updated,is_actual,district,price_sotka
1,623819209,https://www.avito.ru/uvarovka/zemelnye_uchastk...,"Участок 12 сот. (СНТ, ДНП)",199999,Земли сельскохозяйственного назначения,12.0,2024-02-29 17:45:26,Частное лицо (фильтр),Московская область,Уваровка,"Московская обл., Можайский г.о., коттеджный по...",В будние дни звонить ПОСЛЕ 18:00. До этого вре...,Продам,"55.66884943217913, 35.57753456303382",-,2023-11-24 14:51:40,2024-02-22 14:58:27,11,Можайский Городской Округ,16667.0


In [450]:
'Сгруппируем объявления из Авито по региону, району, категории. Оставим регионы с более 10 объявлениями'

avito_lp_by_district =  avito_add_living.groupby(['region', 'district', 'land_cat'], as_index=False) \
            .agg(add_count=("link", "count"),
                q25_pr_sot=("price_sotka", lambda x: np.quantile(x, 0.25).round(0)),
                median_pr_sot=("price_sotka", lambda x: np.median(x).round(0)),
                q75_pr_sot=("price_sotka", lambda x: np.quantile(x, 0.75).round(0))) \
            .sort_values(['region', 'median_pr_sot'], ascending=[True, False]) \
            .query('add_count >= 10')

In [452]:
'Сгруппируем объявления из Авито по региону, категории. Оставим регионы с более 50 объявлениями'

avito_lp_by_region =  avito_add_living.groupby(['region', 'land_cat'], as_index=False) \
            .agg(add_count=("link", "count"),
                q25_pr_sot=("price_sotka", lambda x: np.quantile(x, 0.25).round(0)),
                median_pr_sot=("price_sotka", lambda x: np.median(x).round(0)),
                q75_pr_sot=("price_sotka", lambda x: np.quantile(x, 0.75).round(0))) \
            .sort_values(['region', 'median_pr_sot'], ascending=[True, False]) \
            .query('add_count >= 50')

In [453]:
avito_lp_by_district.shape

(2069, 7)

In [456]:
avito_lp_by_region.shape

(158, 6)

In [457]:
'Сохраняем итоговую таблицу с группировкой по районам в csv'

avito_lp_by_district.to_csv('avito_stats_by_district.csv', index=False)

In [458]:
'Сохраняем итоговую таблицу с группировкой по регионам в csv'

avito_lp_by_region.to_csv('avito_stats_by_region.csv', index=False)